# **Cluster**

## Objectives

* Fit and evaluate a cluster model to group similar data
* Understand the profile for each cluster


## Inputs

* outputs/datasets/collection/airplane_performance_study.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in their respective notebooks.

## Outputs

* Cluster Pipeline
* Train Set
* Most important features to define a cluster plot
* Clusters Profile Description
* Cluster Silhouette


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the parent of current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

We are dropping:
* Model, Company because these are only meta data/identifiers
* 'Multi_Engine' and 'Engine_Type' becasue we want to evaluate against these
* THR and SHP both because they are difficult to interpret
* Hmax_(One) and ROC_(One) becasue since they ONLY apply to the Airplanes with a "True" under the "Multi Engine"-feature it would therefore skew the result. A run that included Hmax_(One) and ROC_(One) indeed show Hmax_(One) on top as the most important feature however ROC_(One) was even listed.

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/airplane_performance_study.csv")
      .drop(['Model', 'Company', 'Multi_Engine', 'Engine_Type', 'THR', 'SHP', 'Hmax_(One)', 'ROC_(One)'], axis=1)
      )
print(df.shape)
df.head(3)


---

# Cluster Pipeline with all data

##  ML Cluster Pipeline

In [ ]:
print(df.dtypes)

In [ ]:
# Convert boolean columns to strings
df['TP_mods'] = df['TP_mods'].replace({True: 'Yes', False: 'No'})

# Convert integer columns to strings (if they represent categories)
#df['Engine_Type'] = df['Engine_Type'].astype(str)

# Check the updated data types
print(df.dtypes)


We will use K-means since this model is ideal for numerical continuous data and the majority of our data is numerical continuous. However it is mixed (both categorical and numeric continuous) which could warrant another method like K-modes or K-prototypes that are better suited for mixed data set.

The Threshold is set to 0.5 which is a very low (values are typically 0.85 - 0.9) which is first of all since our values are relatively normally distributed and we shall see that even this low threshold retains only two features! We have experimented with many different threshold values without seeing any significantly difference on n_components.

In [7]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.encoding import OrdinalEncoder
from feature_engine.selection import SmartCorrelatedSelection

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# PCA
from sklearn.decomposition import PCA

# ML algorithm
from sklearn.cluster import KMeans


def PipelineCluster():
    """
    Creates a clustering pipeline for feature engineering, scaling, and dimensionality reduction.

    This function constructs a machine learning pipeline that performs the following steps:
    1. Applies an Ordinal Encoder to specified categorical variables (`TP_mods` and `Engine_Type`) 
       using an arbitrary encoding method.
    2. Utilizes Smart Correlated Selection to filter out features based on correlation, 
       removing those that are highly correlated with a threshold of 0.5.
    3. Scales the features using StandardScaler to standardize the dataset to have a mean of 0 
       and a standard deviation of 1.
    4. Performs Principal Component Analysis (PCA) to reduce the dimensionality of the dataset 
       to 50 components, aiding in visualization and reducing noise.
    5. Applies KMeans clustering to identify 50 clusters in the transformed feature space.

    Returns:
        Pipeline: A scikit-learn Pipeline object that can be used for fitting and transforming
        data prior to clustering.

    Categorical variables:
        - TP_mods
        - Engine_Type

    KMeans parameters:
        - n_clusters: 50
        - random_state: 0 (for reproducibility)
    """
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['TP_mods'])),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman",
                                                              threshold=0.5, selection_method="variance")),

        ("scaler", StandardScaler()),

        ("PCA", PCA(n_components=50, random_state=0)),

        ("model", KMeans(n_clusters=50, random_state=0)),


    ])
    return pipeline_base


## Principal Component Analysis (PCA)

In [ ]:
df.head(10)

We only recieve 2 components. Since the components is 0% and 100% the elbow method (select a number of components around a variance of 80%.) is rendered useless. Other dimensionality reduction techniques like t-SNE or UMAP could be considered.

The "SmartCorrelatedSelection" reduces the number of rows (dimensionality reduction) depending on what threshold value we set! https://feature-engine.trainindata.com/en/latest/user_guide/selection/SmartCorrelatedSelection.html

We are not concerned with the warnings below.

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_pca = Pipeline(pipeline_cluster.steps[:-2])
df_pca = pipeline_pca.fit_transform(df)

print(df_pca.shape,'\n', type(df_pca))

Apply PCA separately to the scaled data

In [ ]:
print(df_pca[:10])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

n_components = 2 # Adapted this to two since this value cannot be higher than 
# the number of retained columns in the df_pca which was only three with a 
# threshold value as low as 0,6


def pca_components_analysis(df_pca, n_components):
    """
    Performs Principal Component Analysis (PCA) on the provided dataset and visualizes the explained variance.

    This function fits a PCA model to the input DataFrame, transforms the data, 
    and calculates the explained variance ratio for each principal component. 
    It then plots the explained variance ratio and accumulated variance as a line plot.

    Parameters:
        df_pca (DataFrame): A pandas DataFrame containing the standardized data to be analyzed by PCA.
        n_components (int): The number of principal components to retain, which should be less than or 
                            equal to the number of features in `df_pca`.

    Outputs:
        Prints the total percentage of variance explained by the specified number of components 
        and displays a line plot showing the explained variance ratio and accumulated variance.

    Notes:
        - Ensure that `df_pca` is preprocessed and suitable for PCA (e.g., scaled if necessary).
        - The number of components must be chosen based on the retained columns in `df_pca`.
    """
    pca = PCA(n_components=n_components).fit(df_pca)
    x_PCA = pca.transform(df_pca)  # array with transformed PCA

    ComponentsList = ["Component " + str(number)
                      for number in range(n_components)]
    dfExplVarRatio = pd.DataFrame(
        data=np.round(100 * pca.explained_variance_ratio_, 3),
        index=ComponentsList,
        columns=['Explained Variance Ratio (%)'])

    dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum(
    )

    PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum(
    )

    print(
        f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
    plt.figure(figsize=(9, 6))
    sns.lineplot(data=dfExplVarRatio,  marker="o")
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0, 110, 10))
    plt.show()


pca_components_analysis(df_pca=df_pca, n_components=n_components)


In [ ]:
pca_components_analysis(df_pca=df_pca,n_components=2)

The Plot with Accumulated CVariance Ratio incldue only 2 components (n_components=2) and we select to retain both thesse since our main purpos with this 
analysis is interpretability and we want to retain as much information as possible for this.

In [ ]:
def PipelineCluster():
    """
    Please see doc string for the first appearance of the function "PipelineCluster" higher up 
    in this notebook. Note that the input varies between the different instances of this function.
    """
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['TP_mods'])),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman",
                                                              threshold=0.5, selection_method="variance")),

        ("scaler", StandardScaler()),

        # we update n_components to 2
        ("PCA", PCA(n_components=2, random_state=0)),

        ("model", KMeans(n_clusters=50, random_state=0)),


    ])
    return pipeline_base


PipelineCluster()


## Elbow Method and Silhouette Score

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_analysis = Pipeline(pipeline_cluster.steps[:-1])
df_analysis = pipeline_analysis.fit_transform(df)

print(df_analysis.shape,'\n', type(df_analysis))

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(KMeans(random_state=0), k=(1,11)) # 11 is not inclusive, it will plot until 10
visualizer.fit(df_analysis) 
visualizer.show() 
plt.show()

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

# 6 is not inclusive, it will stop at 5
n_cluster_start, n_cluster_stop = 2, 6

print("=== Average Silhouette Score for different number of clusters ===")
visualizer = KElbowVisualizer(KMeans(random_state=0), k=(
    n_cluster_start, n_cluster_stop), metric='silhouette')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()
print("\n")


for n_clusters in np.arange(start=n_cluster_start, stop=n_cluster_stop):

    print(f"=== Silhouette plot for {n_clusters} Clusters ===")
    visualizer = SilhouetteVisualizer(estimator=KMeans(n_clusters=n_clusters, random_state=0),
                                      colors='yellowbrick')
    visualizer.fit(df_analysis)
    visualizer.show()
    plt.show()
    print("\n")


The Elbow Method gives a cluster size of two or three. The Silhouette shadow plots appear to clearly point towards 2 clusters so this will be our choice. With two clusters both reaches past the mean line.

In [ ]:
def PipelineCluster():
    """
    Please see doc string for the first appearance of the function "PipelineCluster" higher up 
    in this notebook. Note that the input varies between the different instances of this function.
    """
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['TP_mods'])),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman",
                                                              threshold=0.5, selection_method="variance")),

        ("scaler", StandardScaler()),

        ("PCA", PCA(n_components=2, random_state=0)),

        # we update n_clusters to 2
        ("model", KMeans(n_clusters=2, random_state=0)),


    ])
    return pipeline_base


PipelineCluster()


## Fit Cluster Pipeline

Quick recap of our data for training cluster pipeline

In [ ]:
X = df.copy()
print(X.shape)
X.head(3)

Fit Cluster pipeline

In [ ]:
pipeline_cluster = PipelineCluster()
pipeline_cluster.fit(X)

## Add cluster predictions to dataset

We add a column "Clusters" (with the cluster pipeline predictions) to the dataset

In [ ]:
X['Clusters'] = pipeline_cluster['model'].labels_
print(X.shape)
X.head(3)

In [ ]:
print(f"* Clusters frequencies \n{ X['Clusters'].value_counts(normalize=True).to_frame().round(2)} \n\n")
X['Clusters'].value_counts().sort_values().plot(kind='bar')
plt.show()

In [ ]:
print(df_analysis.shape)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_analysis[:, 0], y=df_analysis[:, 1],  # Original code is: y=df_analysis[:, 1] but that gives an error since it tries to acces the second column of a one column df
                hue=X['Clusters'], palette='Set1', alpha=0.6)
plt.scatter(x=pipeline_cluster['model'].cluster_centers_[:, 1], y=pipeline_cluster['model'].cluster_centers_[:, 1],  # Same here! Original code: cluster_centers_[:, 0]
            marker="x", s=169, linewidths=3, color="black")
plt.xlabel("PCA Component 0")
plt.ylabel("PCA Component 1")
plt.title("PCA Components colored by Clusters")
plt.show()


The fact that the centroid for the smaller cluster (blue) is far away (lateraly) from the actual datapoint (but close longitudinally) might suggest low intrinsic dimensionality that the n_components should have been choosen to 1 instead of 2. Other reasons could be: Poor Cluster Quality, Outliers, High Variability or Feature Scaling Issues.

The alignment of clusters along lines indicates that there are strong linear relationships in the data and therefore perhaps the cluster patterns could be captured with fewer components. The linear relationships probably has the background that the performance features follow similar patterns (what is good for one feature is also good for another feature).The fact that the clusters are clearly spaced from each other, like the smallest (green) cluster suggest that this is a categorical feature that "lifts" all the other features an equal part. Such a candidate would me the TP mods that we have seen stick out before. This is also one of only two categorical values since we have dropped Multi Engine.

In [ ]:
cluster_predictions_with_all_variables = X['Clusters']
cluster_predictions_with_all_variables

## Fit a classifier, where the target is cluster predictions and features remaining variables

We copy `X` to a DataFrame `df_clf`

In [ ]:
df_clf = X.copy()
print(df_clf.shape)
df_clf.head(3)

Split Train and Test sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df_clf.drop(['Clusters'], axis=1),
    df_clf['Clusters'],
    test_size=0.2,
    random_state=0
)

print(X_train.shape, X_test.shape)


Create classifier pipeline steps

In [ ]:
# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithm
from sklearn.ensemble import GradientBoostingClassifier


def PipelineClf2ExplainClusters():
    """
    Constructs a machine learning pipeline for classification tasks with feature selection and scaling.

    The pipeline includes the following steps:
        1. **Ordinal Encoding**: Transforms categorical variables into ordinal format using
           an arbitrary encoding method for the variable 'TP_mods'.
        2. **Smart Correlation Selection**: Selects features based on their correlation to the target
           variable using the Spearman method, with a threshold of 0.5 and a variance-based selection method.
        3. **Standard Scaling**: Standardizes the features by removing the mean and scaling to unit variance.
        4. **Feature Selection**: Applies SelectFromModel using a Gradient Boosting Classifier to select
           the most important features.
        5. **Model Training**: Finally, a Gradient Boosting Classifier is used as the main model for training.

    Returns:
        Pipeline: A scikit-learn Pipeline object configured with the specified preprocessing and model
        training steps.
    """
    pipeline_base = Pipeline([
        ("OrdinalCategoricalEncoder", OrdinalEncoder(encoding_method='arbitrary',
                                                     variables=['TP_mods'])),

        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman",
                                                              threshold=0.5, selection_method="variance")),

        ("scaler", StandardScaler()),

        ("feat_selection", SelectFromModel(
            GradientBoostingClassifier(random_state=0))),

        ("model", GradientBoostingClassifier(random_state=0)),

    ])
    return pipeline_base


PipelineClf2ExplainClusters()


Fit the classifier to the training data

In [ ]:
pipeline_clf_cluster = PipelineClf2ExplainClusters()
pipeline_clf_cluster.fit(X_train, y_train)

## Evaluate classifier performance on Train and Test Sets

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, pipeline_clf_cluster.predict(X_train)))

In [ ]:
print(classification_report(y_test, pipeline_clf_cluster.predict(X_test)))

## Assess the most important Features that define a cluster

The most important feature being TP mods is rather disappointing since this is a trivial result from an Aircraft Design point of view.

In [ ]:
# after data cleaning and feature engineering, the feature space changes

# how many data cleaning and feature engineering steps does your pipeline have?
data_cleaning_feat_eng_steps = 2
columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_clf_cluster.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[pipeline_clf_cluster['feat_selection'].get_support()],
    'Importance': pipeline_clf_cluster['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# reassign best features in importance order
best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features} \n")
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()


We will store the best_features to use at a later stage.

In [ ]:
best_features_pipeline_all_variables = best_features
best_features_pipeline_all_variables

## Cluster Analysis

Load function that plots a table with description for all Clusters

In [33]:

def DescriptionAllClusters(df, decimal_points=3):
    """
    Generates a summary description for each cluster in the provided DataFrame.

    This function iterates through each unique cluster in the input DataFrame, 
    subsets the data for each cluster, and computes a detailed description using 
    the `Clusters_IndividualDescription` function. The results are compiled into 
    a new DataFrame, which is returned.

    Parameters:
        df (DataFrame): A pandas DataFrame containing data with a 'Clusters' column 
                        that indicates the cluster assignment for each row.
        decimal_points (int, optional): The number of decimal points to display in 
                                         the summary statistics. Default is 3.

    Returns:
        DataFrame: A DataFrame containing the summary description of each cluster, 
                    indexed by cluster.

    Notes:
        - The input DataFrame must include a column named 'Clusters' to categorize 
          the data into distinct groups.
        - The function assumes that the `Clusters_IndividualDescription` function 
          is defined elsewhere and handles the individual cluster description logic.
    """
    DescriptionAllClusters = pd.DataFrame(
        columns=df.drop(['Clusters'], axis=1).columns)
    # iterate on each cluster , calls Clusters_IndividualDescription()
    for cluster in df.sort_values(by='Clusters')['Clusters'].unique():

        EDA_ClusterSubset = df.query(
            f"Clusters == {cluster}").drop(['Clusters'], axis=1)
        ClusterDescription = Clusters_IndividualDescription(
            EDA_ClusterSubset, cluster, decimal_points)
        DescriptionAllClusters = DescriptionAllClusters.append(
            ClusterDescription)

    DescriptionAllClusters.set_index(['Cluster'], inplace=True)
    return DescriptionAllClusters


def Clusters_IndividualDescription(EDA_Cluster, cluster, decimal_points):
    """
    Generates a descriptive summary for a specified cluster in a DataFrame, including statistics for numerical 
    and categorical variables.

    For each column in the input DataFrame (`EDA_Cluster`):
    - If the column is categorical, the function counts the frequencies of the top three most common categories 
      and calculates their respective percentages.
    - If the column is numerical, it calculates the interquartile range (IQR) using the first (Q1) and third 
      quartiles (Q3), providing a range that captures the most common values.

    Args:
        EDA_Cluster (pd.DataFrame): A DataFrame containing data to analyze, where each row corresponds to an 
                                     instance and each column corresponds to a variable.
        cluster (int or str): An identifier for the specific cluster being described.
        decimal_points (int): The number of decimal places to round the quartile values.

    Returns:
        pd.DataFrame: A DataFrame containing a summary description for the specified cluster, with columns 
                      representing the original variables and a final column indicating the cluster identifier.

    Notes:
        - If a variable has only missing data within the specified cluster, the function will report 
          'Not available' for that variable.
        - Any errors encountered during processing will be printed to the console, along with details about 
          the specific variable and cluster.
    """
    ClustersDescription = pd.DataFrame(columns=EDA_Cluster.columns)
    # for a given cluster, iterate over all columns
    # if the variable is numerical, calculate the IQR: display as Q1 -- Q3.
    # That will show the range for the most common values for the numerical variable
    # if the variable is categorical, count the frequencies and displays the top 3 most frequent
    # That will show the most common levels for the category

    for col in EDA_Cluster.columns:

        try:  # eventually a given cluster will have only missing data for a given variable

            if EDA_Cluster[col].dtypes == 'object':

                top_frequencies = EDA_Cluster.dropna(
                    subset=[col])[[col]].value_counts(normalize=True).nlargest(n=3)
                Description = ''

                for x in range(len(top_frequencies)):
                    freq = top_frequencies.iloc[x]
                    category = top_frequencies.index[x][0]
                    CategoryPercentage = int(round(freq*100, 0))
                    statement = f"'{category}': {CategoryPercentage}% , "
                    Description = Description + statement

                ClustersDescription.at[0, col] = Description[:-2]

            elif EDA_Cluster[col].dtypes in ['float', 'int']:
                DescStats = EDA_Cluster.dropna(subset=[col])[[col]].describe()
                Q1 = round(DescStats.iloc[4, 0], decimal_points)
                Q3 = round(DescStats.iloc[6, 0], decimal_points)
                Description = f"{Q1} -- {Q3}"
                ClustersDescription.at[0, col] = Description

        except Exception as e:
            ClustersDescription.at[0, col] = 'Not available'
            print(
                f"** Error Exception: {e} - cluster {cluster}, variable {col}")

    ClustersDescription['Cluster'] = str(cluster)

    return ClustersDescription


Load a custom function to plot cluster distribution per Variable (absolute and relative levels)

In [34]:
import plotly.express as px


def cluster_distribution_per_variable(df, target):
    """
    The data should have 2 variables, the cluster predictions and
    the variable you want to analyze with, in this case we call "target".
    We use plotly express to create 2 plots:
    Cluster distribution across the target.
    Relative presence of the target level in each cluster.
    """
    df_bar_plot = df.value_counts(["Clusters", target]).reset_index()
    df_bar_plot.columns = ['Clusters', target, 'Count']
    df_bar_plot[target] = df_bar_plot[target].astype('object')

    print(f"Clusters distribution across {target} levels")
    fig = px.bar(df_bar_plot, x='Clusters', y='Count',
                 color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.show(renderer='jupyterlab')

    df_relative = (df
                   .groupby(["Clusters", target])
                   .size()
                   .groupby(level=0)
                   .apply(lambda x:  100*x / x.sum())
                   .reset_index()
                   .sort_values(by=['Clusters'])
                   )
    df_relative.columns = ['Clusters', target, 'Relative Percentage (%)']

    print(f"Relative Percentage (%) of {target} in each cluster")
    fig = px.line(df_relative, x='Clusters', y='Relative Percentage (%)',
                  color=target, width=800, height=500)
    fig.update_layout(xaxis=dict(tickmode='array',
                      tickvals=df['Clusters'].unique()))
    fig.update_traces(mode='markers+lines')
    fig.show(renderer='jupyterlab')


Create a DataFrame that contains best features and Clusters Predictions since we want to analyse the patterns for each cluster.


In [ ]:
df_cluster_profile = df_clf.copy()
df_cluster_profile = df_cluster_profile.filter(items=best_features + ['Clusters'], axis=1)
print(df_cluster_profile.shape)
df_cluster_profile.head(3)

We want also to analyse distribution of Multi Engine and Engine Types.

In [ ]:
df_multi_engine = pd.read_csv("outputs/datasets/collection/airplane_performance_study.csv").filter(['Multi_Engine'])
df_multi_engine['Multi_Engine'] = df_multi_engine['Multi_Engine'].astype('object')
df_multi_engine.head(3)

Also testing for Engine Type albeit not the aim for the exercise.

In [ ]:
df_engine_type = pd.read_csv("outputs/datasets/collection/airplane_performance_study.csv").filter(['Engine_Type'])
df_engine_type['Engine_Type'] = df_engine_type['Engine_Type'].astype('object')
df_engine_type.head(3)

### Cluster profile based on the best features

For Multi Engine.

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile_multi_engine = DescriptionAllClusters(df=pd.concat([df_cluster_profile,df_multi_engine], axis=1), decimal_points=0)
clusters_profile_multi_engine.to_csv(f"outputs/ml_pipeline/cluster_analysis/v1/clusters_profile_multi_engine.csv")
clusters_profile_multi_engine

...and for Engine Type

In [ ]:
pd.set_option('display.max_colwidth', None)
clusters_profile_engine_type = DescriptionAllClusters(df=pd.concat([df_cluster_profile,df_engine_type], axis=1), decimal_points=0)
clusters_profile_engine_type.to_csv(f"outputs/ml_pipeline/cluster_analysis/v1/clusters_profile_engine_type.csv")
clusters_profile_engine_type

The above Cluster Profile is very important since we from this plot can draw conclusions of the patterns.

The description of the clusters tells us not surprisingly that the modification to the Engine Performance (TP_mods) was perfectly clustered and that Airplanes without TP mods are pretty much fifty-fifty when it comes to Multi Engine. Furthermore it shows that the TP mods all most only appear on Piston Engines.

The most important conclusion from this clustering exercise is that TP mods appear to be:
* Exclusively made on piston powered airplanes!
* more common on Multi Engine Airplanes where 2/3 of the TP mods being done on Multi Engines.

Since only one feature has been retained together with that the value of this clustering exercise has proved to be disappointing makes us not continue any furter with a re-fitting of the pipeline.

The reason that this clustering did not yield much valuable insight is most likely becaus the the data set was to homogenous in terms of airplane types and that the airplanes in the data set was quite evenly and continuously distributed in terms of features such as size for example. If we would instead would have dropped the mid sized airplanes or introduced another type of airplanes such as fighter jets or drones we would most likely have experienced a more clear clustering. 

### Clusters distribution across Multi Engine and Engine Type distributions & Relative Percentage of Multi Engine and Engine Type in each cluster

In [ ]:
df_cluster_vs_multi_engine=  df_multi_engine.copy()
df_cluster_vs_multi_engine['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_multi_engine, target='Multi_Engine')
plt.savefig(f"outputs/ml_pipeline/cluster_analysis/v1/cluster_vs_multi_engine.png", bbox_inches='tight', dpi=150) # Save plot

In [ ]:
df_cluster_vs_engine_type=  df_engine_type.copy()
df_cluster_vs_engine_type['Clusters'] = X['Clusters']
cluster_distribution_per_variable(df=df_cluster_vs_engine_type, target='Engine_Type')
plt.savefig(f"outputs/ml_pipeline/cluster_analysis/v1/cluster_vs_engine_type.png", bbox_inches='tight', dpi=150) # Save plot

---

# Push files to Repo


We will generate the following files

* Cluster Pipeline
* Train Set
* Feature importance plot
* Clusters Description
* Cluster Silhouette


In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/cluster_analysis/{version}'

try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)


## Cluster pipeline

In [ ]:
pipeline_cluster

In [ ]:
joblib.dump(value=pipeline_cluster, filename=f"{file_path}/cluster_pipeline.pkl")

## Train Set

In [ ]:
df_cluster_study=df
print(df.shape)
df_cluster_study.head(3)

In [46]:
df_cluster_study.to_csv(f"{file_path}/TrainSet.csv", index=False)

## Most important features plot

These are the features that define a cluster

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance', figsize=(8,4))
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance', figsize=(8,4))
plt.savefig(f"{file_path}/features_define_cluster.png", bbox_inches='tight', dpi=150)

## Cluster silhouette plot

In [ ]:
visualizer = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick')
visualizer.fit(df_analysis)
visualizer.show()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(7,5))
fig = SilhouetteVisualizer(Pipeline(pipeline_cluster.steps[-1:])[0] , colors='yellowbrick', ax=axes)
fig.fit(df_analysis)

plt.savefig(f"{file_path}/clusters_silhouette.png", bbox_inches='tight',dpi=150)

Based on the profile results we can label the two clusters in the following fashion:
* **Cluster 0** represent airplanes that does not have TP mods and is made up by all Engine Types (piston, propjet and jet).
* **Cluster 1** represent airplanes that has TP mods and is almost exclusively made up by Piston powered Airplanes
(Airplane Type: piston) indicating that the TP mods is a feature applicable only on Piston Powered Engines. Furthermore Multi Engine Airplanes represented 2/3 of all TP mods.